# Part 1 : Index Construction

## Data Crawling

In [11]:
from googlesearch import search 
import tldextract  
import requests 
from bs4 import BeautifulSoup 
import csv   
from time import sleep 

# to search 
queries = ["Computer Science", "IITs in India", "Cities of Chattisgarh"]

#Loop for each query
for i in range(len(queries)):
    query = queries[i]
    # Loop for each search result
    count_files = 20
    urls = []
    for url in search(query,num=25, start= 1, stop=50, pause=2): 
        urls.append(url)
        
    for url in urls:
        print(url)
        try:
            respons = requests.get(url, timeout=5) #Response for the current URL
        except:
            continue
        sleep(0.5)
        soup = BeautifulSoup(respons.content, 'html5lib')
        paras=soup.find_all('p')  #Finding all the paragraph within the response
        para = "" #Initializing a empty paragraph
        
        count = 2 #To insure that file must contains 2 paragraph
        flag = 0
        
        for p in paras:   # Loop though each paragraph in response and append the contnet of that paragraph to  "para" so that we can save it as a continue paragraph
            if p!="":
                para += p.get_text()
                count-=1
            if (count==0):
                flag=1
                break
                
        if(flag):  #Perfom writing to file onlw when there is something in para
            ext_url = tldextract.extract(url)  #parsing the URL 
            domin_name = ext_url.domain   #Get the domain name from the parsed URL so that we can use it as a fine name

            with open(f'./d{21-count_files}_q{i+1}.txt', mode='wt', encoding='utf-8') as file:  #Wtite "para" to a txt file whose filename is a domain name of current URL
                file.write(para)
            count_files-=1
        
        if count_files==0:
            break

https://www.britannica.com/science/computer-science
https://www.edx.org/course/subject/computer-science
https://www.khanacademy.org/computing/computer-science
https://www.internationalstudent.com/study-computer-science/what-is-computer-science/
https://www.youtube.com/watch/Tzl0ELY_TiM
https://www.coursera.org/browse/computer-science
https://www.topuniversities.com/courses/computer-science-information-systems/guide
https://www.timeshighereducation.com/student/what-to-study/computer-science
https://www.computerscience.org/
https://arxiv.org/archive/cs
https://www.seas.harvard.edu/computer-science
https://engineering.buffalo.edu/computer-science-engineering.html
https://www.timeshighereducation.com/student/subjects/what-can-you-do-computer-science-degree
https://engineering.virginia.edu/departments/computer-science
https://www.codecademy.com/learn/paths/computer-science
https://www.ou.edu/coe/cs
https://www.usnews.com/best-graduate-schools/top-science-schools/computer-science-rankings
ht

## Inverted Index

Associate a collection of terms (lexicon) with the documents that contain those terms.
The data structure is much more dense than a Document Term Matrix.

In [12]:
# Collection of documents (corpus)

docs = []
for i in range(3):
    doc = []
    for j in range(20):
        with open(f'./d{j+1}_q{i+1}.txt') as f:
            doc.append(f.read())
    docs.append(doc)

docs

[['Our editors will review what you’ve submitted and determine whether to revise the article.Computer science,  the study of computers and computing, including their theoretical and algorithmic foundations, hardware and software, and their uses for processing information. The discipline of computer science includes the study of algorithms and data structures, computer and network design, modeling data and information processes, and artificial intelligence. Computer science draws some of its foundations from mathematics and engineering and therefore incorporates techniques from areas such as queueing theory, probability and statistics, and electronic circuit design. Computer science also makes heavy use of hypothesis testing and experimentation during the conceptualization, design, measurement, and refinement of new algorithms, information structures, and computer architectures.',
  'Real college courses from Harvard, MIT, and more of the world’s leading universities.Nationally in Compu

In [18]:
# Gather the set of all unique terms

unique_terms = {term for doc_list in docs  for doc in doc_list for term in doc.split()}
unique_terms

{'¢*F',
 'Rø³ýÈ!ŠÄ“¶ß-\x13µ\x10�',
 ',\x0e-Õ|7Á÷\x0e¡r(]ÍUÉ:\x1aGè^C@üp',
 'h}n—\x026Eàùï’(´°\x03UJmD�ÆÛo¿Œ�\x06^Q29«þ×',
 'institutions"—one',
 "¸Lù×«P\x0ezM='²*\xadé°¢ÛÐ–",
 'IB[©T³#êÖDµÈtÈ“¡œQ\x12O�\x08aÔ\x14²A„æýùP7ó,n\x1bZ\x04\x1b«ßèÄmå1–eM5L9¤öË3xµ/T²²',
 '0Æ\x15OgùÚ`g\x03¦Ö·1SµI>•J/\x06Åz½Ÿ¯H¼I±B8\x02º8”¸Øìfì@\x12?æë\x14¬',
 '€8tZ¸',
 'yõÐƒ‹‹àcvÓ\x07\x14MYzÀ¬üv1\x13„Æ?1ÄÔûu=',
 '¼2Ì3¦\x0fö\x0fÇ)&E',
 'ÀP\x1a×5½°\x11à\x14&0gë›\x01!',
 'ÂXè\x02\x02\x02\xad=õfI�4‘\x07=\x05/¡©ûk…\x13à}ºÕ>Ø\x0eíTÝJKž™„qAÊŸ©',
 'Y�w0ÇÙ,(áÏãŸ\x07]ê',
 'K[†\x0eÏÃç×I+y\xad8W\x7f};pÔ‡§/%³$ž',
 'Ê\x15©a…‹-\x14¥0',
 'intelligence.',
 'ov*èUé�Æ/÷¯\xadtV!c¶',
 '3”',
 'such',
 '\x06¸¬<=Ø0L¨_êû|žN\x14]\x07',
 'Æ^�\x1aw\x03šE¾Ÿ†éÍùd[Ímuis¤¤/<\x1a¾À;ìÝ-âŒ\x07�',
 'JN¯Â%\x065ÄT;\\ôç8[÷OªÔÃñ',
 '³Faì,í™ò',
 "Ä¸\x1bs\x1bh'wxä¬ð�_@\x10»ÞÉ%¶‘¹$\x06g*M=núˆ=D�•0H~\x06",
 '¨«\x16ÀdA0\x03ð\x04\x7f|“ˆU·›Ð—-<�Óµk·\x12•`±',
 'Q¾ëh‰�Œ\x02¹±7{\x14s¥bõ°Ò~K',
 'ËÜ\x03·W›±ò˜)l<ú\x0eÉB^\x17ºÌ¿Vàé–¸£_Ûd4…¹¤ù@.ËÙ›',
 '³°¼ÃMwÿ/jp\x10

In [19]:
# Construct an inverted index
# here as a Python dictionary for ease of interpretability

inverted_index = {}
for query_number in range(3):
    for doc_number in range(20):
    doc = docs[query_number][doc_number]
        for term in doc.split():
            if term in inverted_index:
                inverted_index[term].add(f"d{doc_number}_q{query_number}")
            else: inverted_index[term] = {f"d{doc_number}_q{query_number}"}

inverted_index

AttributeError: 'list' object has no attribute 'split'